In [ ]:
import numpy as np
import pandas as pd
import os
import scipy.io
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
import scipy.io
import time

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install pyreadr
import pyreadr

     |████████████████████████████████| 419kB 5.2MB/s 


In [ ]:
train_dat = pyreadr.read_r('/gdrive/MyDrive/STAT5243/data/feature_train.RData')
test_dat = pyreadr.read_r('/gdrive/MyDrive/STAT5243/data/feature_test.RData')

train_dat = list(train_dat.items())[0][1]
test_dat = list(test_dat.items())[0][1]

In [ ]:
x_train = train_dat.iloc[:,:-1]
y_train = train_dat.iloc[:,-1]
x_test = test_dat.iloc[:,:-1]
y_test = test_dat.iloc[:,-1]

In [ ]:
def create_nn(n1, n2, n3, optimizer='adam'):
  # Initialize the constructor
  model = Sequential()
  # Add input and hidden layers
  model.add(Dense(n1, activation='relu', input_shape=[6006]))
  model.add(Dense(n2, activation='relu'))
  if n3 != None:
    model.add(Dense(n3, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #compile model
  model.compile(loss='binary_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy', 'AUC'])
  return model

In [ ]:
nn = KerasClassifier(build_fn=create_nn, verbose=1)
n1 = [9, 7]
n2 = [7, 5, 3]
n3 = [None, 3]

In [ ]:
param_grid = dict(n1=n1, n2=n2, n3=n3)
grid = GridSearchCV(estimator=nn, param_grid=param_grid, n_jobs=-1, cv=5)

In [ ]:
grid_result = grid.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


75/75 [==============================] - 1s 4ms/step - loss: 3.1647 - accuracy: 0.7065 - auc: 0.5231


In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, sd, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, sd, param))

Best: 0.805833 using {'n1': 9, 'n2': 7, 'n3': 3}
0.678333 (0.248651) with: {'n1': 9, 'n2': 7, 'n3': None}
0.805833 (0.009807) with: {'n1': 9, 'n2': 7, 'n3': 3}
0.689167 (0.240512) with: {'n1': 9, 'n2': 5, 'n3': None}
0.805833 (0.009807) with: {'n1': 9, 'n2': 5, 'n3': 3}
0.805833 (0.009807) with: {'n1': 9, 'n2': 3, 'n3': None}
0.805833 (0.009807) with: {'n1': 9, 'n2': 3, 'n3': 3}
0.805833 (0.009807) with: {'n1': 7, 'n2': 7, 'n3': None}
0.682500 (0.245609) with: {'n1': 7, 'n2': 7, 'n3': 3}
0.805833 (0.009807) with: {'n1': 7, 'n2': 5, 'n3': None}
0.805833 (0.009807) with: {'n1': 7, 'n2': 5, 'n3': 3}
0.682500 (0.245609) with: {'n1': 7, 'n2': 3, 'n3': None}
0.805833 (0.009807) with: {'n1': 7, 'n2': 3, 'n3': 3}


In [ ]:
n1, n2, n3 = grid_result.best_params_.values()

In [ ]:
def create_nn(n1=n1, n2=n2, n3=n3, optimizer='adam'):
  # Initialize the constructor
  model = Sequential()
  # Add input and hidden layers
  model.add(Dense(n1, activation='relu', input_shape=[6006]))
  model.add(Dense(n2, activation='relu'))
  if n3 != None:
    model.add(Dense(n3, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #compile model
  model.compile(loss='binary_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy', 'AUC'])
  return model

In [ ]:
def create_nn(n1=n1, n2=n2, n3=n3, optimizer='adam'):
  # Initialize the constructor
  model = Sequential()
  # Add input and hidden layers
  model.add(Dense(n1, activation='relu', input_shape=[6006]))
  model.add(Dense(n2, activation='relu'))
  if n3 != None:
    model.add(Dense(n3, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #compile model
  model.compile(loss='binary_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy', 'AUC'])
  return model

In [ ]:
nn = KerasClassifier(build_fn=create_nn, verbose=1)
batch_size = [16, 32, 64, 128]
epochs = [10, 25, 50, 100]

In [ ]:
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=nn, param_grid=param_grid, n_jobs=-1, cv=5)

In [ ]:
grid_result = grid.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/50
75/75 [==============================] - 1s 4ms/step - loss: 6.5433 - accuracy: 0.6958 - auc: 0.4847
Epoch 2/50
75/75 [==============================] - 0s 4ms/step - loss: 0.6693 - accuracy: 0.8094 - auc: 0.5085
Epoch 3/50
75/75 [==============================] - 0s 4ms/step - loss: 0.6501 - accuracy: 0.8061 - auc: 0.5151
Epoch 4/50
75/75 [==============================] - 0s 4ms/step - loss: 0.6331 - accuracy: 0.8028 - auc: 0.4948
Epoch 5/50
75/75 [==============================] - 0s 4ms/step - loss: 0.6165 - accuracy: 0.8058 - auc: 0.5053
Epoch 6/50
75/75 [==============================] - 0s 3ms/step - loss: 0.6013 - accuracy: 0.8085 - auc: 0.5003
Epoch 7/50
75/75 [==============================] - 0s 4ms/step - loss: 0.5869 - accuracy: 0.8119 - auc: 0.4968
Epoch 8/50
75/75 [==============================] - 0s 3ms/step - loss: 0.5746 - accuracy: 0.8129 - auc: 0.5031
Epoch 9/50
75/75 [==============================] - 0s 4ms/step - loss: 0.5648 - accuracy: 0.8110 - auc:

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, sd, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, sd, param))

Best: 0.806667 using {'batch_size': 32, 'epochs': 50}
0.805833 (0.009807) with: {'batch_size': 16, 'epochs': 10}
0.805833 (0.009807) with: {'batch_size': 16, 'epochs': 25}
0.805833 (0.009807) with: {'batch_size': 16, 'epochs': 50}
0.805833 (0.009807) with: {'batch_size': 16, 'epochs': 100}
0.805833 (0.009807) with: {'batch_size': 32, 'epochs': 10}
0.805833 (0.009807) with: {'batch_size': 32, 'epochs': 25}
0.806667 (0.010155) with: {'batch_size': 32, 'epochs': 50}
0.805833 (0.009807) with: {'batch_size': 32, 'epochs': 100}
0.805833 (0.009807) with: {'batch_size': 64, 'epochs': 10}
0.805833 (0.009807) with: {'batch_size': 64, 'epochs': 25}
0.805833 (0.009807) with: {'batch_size': 64, 'epochs': 50}
0.805833 (0.009807) with: {'batch_size': 64, 'epochs': 100}
0.805833 (0.009807) with: {'batch_size': 128, 'epochs': 10}
0.805833 (0.009807) with: {'batch_size': 128, 'epochs': 25}
0.805833 (0.009807) with: {'batch_size': 128, 'epochs': 50}
0.805833 (0.009807) with: {'batch_size': 128, 'epochs':

In [ ]:
grid.best_estimator_

In [ ]:
best_model = Sequential()
best_model.add(BatchNormalization(input_shape=[6006], momentum=0.80))
best_model.add(Dense(1024, activation='relu'))
best_model.add(Dropout(0.5))
best_model.add(Dense(512, activation='relu'))
best_model.add(Dropout(0.5))
best_model.add(Dense(256, activation='relu'))
best_model.add(Dropout(0.3))
best_model.add(Dense(128, activation='relu'))
best_model.add(Dropout(0.3))
best_model.add(Dense(64, activation='relu'))
best_model.add(Dropout(0.2))
best_model.add(Dense(32, activation='relu'))
best_model.add(Dropout(0.2))
best_model.add(Dense(16, activation='relu'))
best_model.add(Dropout(0.1))
best_model.add(Dense(8, activation='relu'))
best_model.add(Dropout(0.1))
best_model.add(Dense(1, activation='sigmoid'))

In [ ]:
#compile model
best_model.compile(loss='binary_crossentropy', 
                   optimizer='adam', 
                   metrics=['accuracy', 'AUC'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=20,
    mode='max',
    restore_best_weights=True)

In [ ]:
y_train = y_train.astype('int')

In [ ]:
start_time = time.time()
best_model.fit(x_train, 
               y_train, 
               batch_size=32, 
               epochs=100, 
               validation_split=0.15,
               callbacks = [early_stopping])
print('Training on 2400 images with train-val split of 85-15 takes %s seconds.'%round((time.time()-start_time),3))

Epoch 1/100
64/64 [==============================] - 7s 89ms/step - loss: 0.8816 - accuracy: 0.7499 - auc: 0.4810 - val_loss: 0.6809 - val_accuracy: 0.7944 - val_auc: 0.5223
Epoch 2/100
64/64 [==============================] - 6s 95ms/step - loss: 0.7757 - accuracy: 0.7673 - auc: 0.5026 - val_loss: 0.6595 - val_accuracy: 0.7944 - val_auc: 0.5063
Epoch 3/100
64/64 [==============================] - 5s 81ms/step - loss: 0.6750 - accuracy: 0.7938 - auc: 0.5178 - val_loss: 0.5337 - val_accuracy: 0.7944 - val_auc: 0.5323
Epoch 4/100
64/64 [==============================] - 5s 77ms/step - loss: 0.6169 - accuracy: 0.8043 - auc: 0.5032 - val_loss: 0.5789 - val_accuracy: 0.7944 - val_auc: 0.5449
Epoch 5/100
64/64 [==============================] - 5s 80ms/step - loss: 0.5804 - accuracy: 0.8009 - auc: 0.5128 - val_loss: 0.5338 - val_accuracy: 0.7944 - val_auc: 0.5519
Epoch 6/100
64/64 [==============================] - 6s 88ms/step - loss: 0.5453 - accuracy: 0.8058 - auc: 0.5437 - val_loss: 0.53

In [ ]:
y_test = y_test.astype('int')

In [ ]:
start_time = time.time()
best_model.evaluate(x_test, y_test, batch_size=32, verbose=1)
print('Prediction on 600 images takes %s seconds.'%round((time.time()-start_time),3))

19/19 [==============================] - 0s 20ms/step - loss: 1.0841 - accuracy: 0.8300 - auc: 0.8147
Prediction on 600 images takes 0.562 seconds.


In [ ]:
tf.keras.models.save_model(best_model, '/gdrive/MyDrive/STAT5243/data/neural_network', save_format='h5')

In [ ]:
best_model = tf.keras.models.load_model('/gdrive/MyDrive/STAT5243/data/neural_network')

In [ ]:
best_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 9)                 54063     
_________________________________________________________________
dense_23 (Dense)             (None, 7)                 70        
_________________________________________________________________
dense_24 (Dense)             (None, 3)                 24        
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 4         
Total params: 54,161
Trainable params: 54,161
Non-trainable params: 0
_________________________________________________________________
